
# 核心思想
FlashAttention 的核心在于减少 GPU HBM (显存) 的读写次数。标准 Attention 需要计算并存储 $N \times N$ 的巨大矩阵，而 FlashAttention 通过分块计算，将计算过程限制在快速的 SRAM 中。
# 前向传播

已知量：Q, K, V -> [N, D];

待求量：O -> [N, D]; M, LSE -> [N,]

计算公式：

$O = P * V$ -> [N, D] = [N, N] * [N, D]

$P = Softmax(S)$;  

$S = \frac{Q * K^T}{\sqrt{D}}$

Online Softmax:

$P = \frac{exp(S-M)}{\sum{exp{S-M}}}$


基本计算流 (The "Fixed Q, Flowing K/V" Strategy):

并行维度：Grid 主要按 $N$ 维度（Sequence Length）划分。每个 Program ID (PID) 负责计算一个 BLOCK_SIZE_QO 大小的 Q 块。

Triton中的grid有两个维度，第一个维度为Q的BLOCK ID，第二个维度为Head ID：

```python
grid = lambda META: (
    triton.cdiv(N, META['BLOCK_SIZE_QO']),
    B * H
)
```

驻留 (Resident)：当前 PID 加载对应的 Q 块 到 SRAM 中。在整个 Inner Loop 过程中，这个 Q 块保持不变。

流动 (Scanning)：K 块 和 V 块 从 HBM 中被分批加载到 SRAM（滑动窗口）。

计算：Q 块与当前的 K 块计算分数，更新 Softmax，与 V 块相乘，累加到输出 O 中，然后丢弃当前的 K/V，加载下一块。

关于MASK：Q只能看到它和它之前的K，对于小于Q的K来说，不需要MASK，对于和Q有重叠的K来说，这个单独的BLOCK里需要MASK。

比如现在的Q BLOCK是Q的64~127行，那么0~63的K不需要MASK,64~127的K需要MASK。

具体实现是根据是否需要MASK，来改变K和V的下界和上界。比如我现在需要计算64~127的Q块，如果需要MASK，那么就只加载[64, 127]的K和V，并且进行mask处理：

```python
causal_mask = offsets_QO_N[:, None] >= (offsets_KV_N[None, :])
S += tl.where(causal_mask, 0, -1.0e6)
```

如果不需要MASK，那么就只加载，[0, 63]的K和V，这时直接计算，不需要mask处理，至于128及以上的，直接不加载。

具体流程：

根据是否需要mask，确定K和V的加载上下界。然后遍历界内：

1. 固定在SRAM中的是$Q_i%，现在遍历到的是$K_j$和$V_j$，计算$S_{ij} = Q_i * K_j^T$ # shape: [BLOCK_SIZE_QO, BLOCK_SIZE_KV]

2. 更新全局最大值，$M_{new} = max(max(S_{ij}), M_{old})$ # shape: [BLOCK_SIZE_QO]

3. 计算修正因子, $\alpha=exp(M_{old} - M_{new})$

4. 计算P，$P = exp(S - M_{new})$

5. 更新分母和LSE, $L_{new} = sum(P, axis=1)$, $LSE = LSE * \alpha + L_{new}$

6. 更新输出O， $O = O * \alpha + P * V$

7. 更新全局最大值，$M = M_{new}$

8. 移动K和V的指针

代码：

```python
# 遍历K和V的分块
    for start_kv in range(low, high, BLOCK_SIZE_KV):
        # 编译器优化提示
        start_kv = tl.multiple_of(start_kv, BLOCK_SIZE_KV)
        
        # KV的行的mask
        mask_KV_N = offsets_KV_N < N

        # 加载K_T，注意因为是转置，所以mask也要转置，mask_KV_N[None, :]
        K_T = tl.load(K_ptr + K_T_offsets, mask=mask_KV_N[None, :], other=0.)

        # Q和K_T相乘
        S = tl.dot(Q, K_T) * scale
        
        if DIAGONAL:
            # offsets_QO_N:[8, 9, 10, 11]
            # offsets_KV_N:[8, 9, 10, 11]
            # 0, 1, 1, 1
            # 0, 0, 1, 1
            # 0, 0, 0, 1
            # 0, 0, 0, 0 
            causal_mask = offsets_QO_N[:, None] >= (offsets_KV_N[None, :])
            S += tl.where(causal_mask, 0, -1.0e6)
        
        # Online Softmax
        # # 需要不断更新每一行的Max值
        # # [BLOCK_SIZE_QO]
        # m_cur_block = tl.max(S, axis=1)

        # # 更新全局最大值
        # # [BLOCK_SIZE_QO]
        # M_new = tl.maximum(m_cur_block, M)
        
        M_new = tl.maximum(M, tl.max(S, axis=1))
        
        # 修正S并计算P
        # P = exp(S - M_new)
        S -= M_new[:, None]
        P = tl.exp2(S)
        
        # 更新分母和L
        # [BLOCK_SIZE_QO]
        L_new = tl.sum(P, axis=1)
        # 根据新的max值，计算修正系数
        alpha = tl.exp2(M - M_new)
        L = L * alpha + L_new
        
        # 更新O的输出
        # O_new = O_old * alpha + P @ V
        V = tl.load(V_ptr + V_offsets, mask=mask_KV_N[:, None], other=0.)
        O = O * alpha[:, None]
        O = tl.dot(P, V, acc=O)
        
        # 更新M [BLOCK_SIZE_QO]
        M = M_new
        
        # 移动指针到下一个Block
        K_T_offsets += BLOCK_SIZE_KV * stride_K_N
        V_offsets += BLOCK_SIZE_KV * stride_V_N
        offsets_KV_N += BLOCK_SIZE_KV
```


# 后向传播

## 公式推导


前向传播的公式为：

$S = scale * (Q * K^T)$

$P = softmax(S) = \frac{exp(S-M)}{\sum{exp(S-M)}}$

$O = P * V$

1. 推导$dV$

$dV = P^T * dO$

2. 推导$dS$

$dP = dO * V^T$

$$
对于一行S=\{s_1, s_2, s_3, s_N\},S_i的梯度dS_{i} = \sum_{j}dP_j * \frac{\partial P_j}{\partial S_i} \\


当i=j时，\frac{\partial P_j}{\partial S_i} = P_j * (1-P_j) \\
当i\neq j时，\frac{\partial P_j}{\partial S_i}  = -P_j*P_i \\
所以，dS_i = \sum_{j} dP_j * (-P_j*P_i)+dP_i*P_i*(1-P_i)=\sum_{j} dP_j * (-P_j*P_i) + dP_i * dP_j \\
= P_i * (dP_i-\sum_{i} dP_j * dP_j)
$$
接下来求$\sum_{i} dP_j * dP_j$

$$
对于第i行的Q，\Delta _{i}=\sum_{k} dP_{ik} * dP_{ik} \\
dP_{ik} = dO_i * V_k^T = \sum _{d} dO_{id} * V_{dk}^T = \sum _{d} dO_{id} * V_{kd} \\
\therefore \Delta _{i}=\sum_{k} P_{ik} \sum_{d}dO_{id}V_{kd}=\sum_{d}dO_{id}\sum_{k}P_{ik}V_{kd} (求和的交换律) \\
=\sum_{d}dO_{id}*O_{id}
\therefore \Delta_{i} = sum(dO * o, axis=-1) \\
\therefore dS = P \circ (dP - \Delta), 其中 \circ 表示逐元素乘法 (Element-wise multiplication)。
$$

3. 推导$dQ, dK$

$$
根据公式S = scale * (Q * K^T)可求得\\
dQ = dS \cdot K \cdot scale\\
dK = dS^T \cdot Q \cdot scale

$$



## Triton实现流程

1. 预处理 (Preprocessing Kernel)

+ 目的：计算辅助变量 $\Delta$。

+ 输入：$O, dO$

+ 操作：Delta = sum(O * dO, axis=-1)

+ 输出：$\Delta$ (Shape: [B, H, N])

2. 反向传播的主逻辑

### 固定Q，流动K和V

此时grid按照K进行分块（BLOCK_SIZE_KV），每个kernel处理一个BLOCK的K和V，把该BLOCK加载到SRAM中，流动Q、dO、LSE、Delta。

首先要重新计算$S = QK^T$ [BLOCK_SIZE_Q, D] * [D, BLOCK_SIZE_KV] = [BLOCK_SIZE_Q, BLOCK_SIZE_KV]

然后计算$P=exp(S-M)$

$dV += P^T \cdot dO$   

shape: [BLOCK_SIZE_KV, BLOCK_SIZE_Q] * [BLOCK_SIZE_Q, D] = [BLOCK_SIZE_KV, D]

$dK += dS^T \cdot Q (dS = P (dP - \Delta))$ 

shape:  [BLOCK_SIZE_KV, BLOCK_SIZE_Q] * [BLOCK_SIZE_Q, D] = [BLOCK_SIZE_KV, D]

### 固定Q，流动K和V

此时grid按照Q进行分块（BLOCK_SIZE_Q），每个kernel处理一个BLOCK的Q，把该BLOCK的Q以及dO,LSE,Delta加载到SRAM中，流动K、V.

重计算P

累积梯度： $dQ += dS \cdot K$



关于固定和流动，以及梯度累加，可以看下图：

<img src="./1.png" alt="Example Image" style="background-color:white;" />
